In [1]:
from pyDatalog import pyDatalog
import logging
from pyDatalog import pyEngine
pyEngine.Logging = False
logging.basicConfig(level=logging.INFO)
pyDatalog.create_terms('link, can_reach,X,Y,Z')
+link(1,2)
+link(2,3)
+link(2,4)
+link(2,5)
+link(5,6)
+link(6,7)
+link(7,2)
link(X,Y) <= link(Y,X)
can_reach(X,Y) <= link(X,Y)
can_reach(X,Y) <= link(X,Z) & can_reach(Z,Y) & (X!=Y)
print (can_reach(Y,3))

Y
-
4
5
7
1
6
2


In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, ForeignKey
from ipywidgets import interact_manual

# define a base class with SQLAlchemy and pyDatalog capabilities
Base = declarative_base(cls=pyDatalog.Mixin, metaclass=pyDatalog.sqlMetaMixin)
# create database in memory
engine = create_engine('sqlite:///:memory:', echo=False)

# open a session on a database, then associate it to the Base class
Session = sessionmaker(bind=engine)
session = Session()
Base.session = session

class Person(Base): # Employee inherits from the Base class
    __tablename__ = 'person' # data are stored in the Employee table

    name = Column(String, primary_key=True)
#     manager_name = Column(String, ForeignKey('employee.name'))
#     salary = Column(Integer)
    def __init__(self, name): 
        # call the initialization method of the Mixin class
        super(Person, self).__init__()
        self.name = name
#         self.manager = manager     # direct manager of the employee, or None
#         self.salary = salary       # monthly salary of the employee
    
    def __repr__(self): # specifies how to display an Employee
        return self.name
    
# now create the table
Base.metadata.create_all(engine)
pyDatalog.create_terms('married, bros, a, c, p, X, Y, Z, K')

# Tywin = Person('Tywin')
# Joanna = Person('Mary')
# Tyrion = Person('Tyrion')
# Jaime = Person('Jaime')
# Cercei = Person('Cercei')
# Tommen = Person('Tommen')
# Myrcella = Person('Myrcella')
# Joffrey = Person('Joffrey')

# +p(Tywin, Tyrion)
# +p(Tywin, Jaime)
# +p(Tywin, Cercei)
# +p(Joanna, Tyrion)
# +p(Joanna, Jaime)
# +p(Joanna, Cercei)
# +p(Jaime, Tommen)
# +p(Jaime, Myrcella)
# +p(Jaime, Joffrey)
# +p(Cercei, Tommen)
# +p(Cercei, Myrcella)
# +p(Cercei, Joffrey)

married(X,Y) <= married(Y,X)
a(X, Y) <= p(X, Y)
a(X, Y) <= p(X, Z) & a(Z, Y)
+p('parent', 'child')
# p(X,Y) <= ~p(Y,X) & (X!=Y)


bros(X, Y) <= p(Z, X) & p(Z, Y)
# p(X, Y) <= c(Y, X) & (X != Y) & ~p(Y, X) & ~c(X, Y) & ~(p(Z, X) & p(Z, Y)) & ~(c(Z, X) & c(Z, Y))
# c(X, Y) <= p(Y, X) & (X != Y) & ~c(Y, X) & ~p(X, Y) & ~(p(Z, X) & p(Z, Y)) & ~(c(Z, X) & c(Z, Y))
married(X,Y) <= p(X,Z) & p(Y,Z) & (X!=Y) & ~bros(X, Y)# c(Z, X) & c(Z, Y) & (X != Y)

persons = []
def add_person(name):
    for person in persons:
        if person.name == name:
            return person
    print("Created " + name)
    result = Person(name)
    persons.append(result)
    return result

@interact_manual
def get_input(parent='', child=''):
    person1 = add_person(parent)
    person2 = add_person(child)
    if person1 != person2 and not [item for item in p(X,Y) if item[0] == person2 and item[1] == person1] \
        and not [item for item in a(X,Y) if item[0] == person2 and item[1] == person1] \
        and not [item for item in a(X,Y) if item[0] == person1 and item[1] == person2] \
        and not [item for item in (p(Z,X) & p(Z,Y) & (X!=Y)) if item[1] == person1 and item[2] == person2]:
        +p(person1, person2)
#     print(p(X, Y))
    print(married(X,Y))

# + has_car(Mary)
# + married(Mary,John)
#print(has_car(X))

# link(X,Y) <= link(Y,X)
# can_reach(X,Y) <= link(X,Y)
# can_reach(X,Y) <= link(X,Z) & can_reach(Z,Y) & (X!=Y)

A Jupyter Widget